# **Промежуточная аттестация по курсу (Python)**

## **ЗАДАНИЕ**

В ячейке ниже представлен код, генерирующий DataFrame, который состоит всего из 1 столбца. Задача состоит в том, чтобы перевести его в one hot вид. Нельзя использовать get_dummies.





---





---



In [55]:
# Инсталляция эмулятора терминала, который можно запустить в Colab
# (ttps://github.com/singhsidhukuldeep/Google-Colab-Shell):

!pip install google-colab-shell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=5bc882e922591bb83c60091dd10e912be2b4b05d31432e82b3cbb66661a15f02
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


In [56]:
from google_colab_shell import getshell

In [57]:
# Запуск эмулятора терминала (для работы с git и П):

getshell()



---



In [1]:
import pandas as pd

import random

In [23]:
lst = ['robot'] * 10
lst += ['human'] * 10
random.shuffle(lst)
data = pd.DataFrame({'whoAmI':lst})

data

,whoAmI
0,human
1,robot
2,robot
3,robot
4,human
5,robot
6,human
7,robot
8,human
9,robot




---



## **РЕШЕНИЕ:**

In [24]:
data['tmp'] = 1
data.set_index([data.index, 'whoAmI'], inplace = True)
data = data.unstack(level=-1, fill_value = 0).astype(int)
data.columns = data.columns.droplevel()
data.columns.name = None

data

,human,robot
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
5,0,1
6,1,0
7,0,1
8,1,0
9,0,1




---





---

